# ENCLAVE
**ENCLAVE** - _Enclaves, Gated Communities and Deforestation Analysis_ - is a spatial analysis project exploring the relationship between the expansion of gated communities and deforestation in Porto Alegre and Santa Cruz do Sul, Brazil.

#### Geospatial data processing

In [9]:
import os
import glob
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
from rasterio.features import shapes
from shapely.geometry import shape

In [11]:
# Set base path. The script assumes it's run from the project's root folder.
BASE_PATH = "."
RAW_DATA_PATH = os.path.join(BASE_PATH, "data", "raw")
PROCESSED_DATA_PATH = os.path.join(BASE_PATH, "data", "processed")

# Define the target municipalities and their acronyms
MUNICIPALITIES = {
    'Santa Cruz do Sul': 'scs',
    'Porto Alegre': 'poa'
}

# Define the target CRS (SIRGAS 2000 / UTM 22S)
TARGET_CRS = 'EPSG:32722'

def process_municipality_data(municipality_name, acronym):
    """
    Processes all raster files for a single municipality, clipping, vectorizing,
    dissolving, and calculating the area of each class.
    
    Args:
        municipality_name (str): The full name of the municipality.
        acronym (str): The desired acronym for the output file.
    """
    print(f"--- Processing data for {municipality_name} ---")

    # Load the municipalities GeoPackage file
    print("Loading municipalities data...")
    try:
        municipios_gdf = gpd.read_file(os.path.join(RAW_DATA_PATH, 'rs_ibge_municipios_2022_pol.gpkg'))
    except Exception as e:
        print(f"Error loading municipalities data: {e}")
        return

    # Filter for the specific municipality and reproject to the target CRS
    municipio_gdf = municipios_gdf[municipios_gdf['NM_MUN'] == municipality_name].to_crs(TARGET_CRS)
    if municipio_gdf.empty:
        print(f"Warning: {municipality_name} not found in the GeoPackage file.")
        return

    # Get a list of all TIF files to be processed
    raster_files = glob.glob(os.path.join(RAW_DATA_PATH, '*.tif'))
    if not raster_files:
        print("No raster files found in the raw data directory.")
        return

    # Loop through each raster file
    for raster_file_path in raster_files:
        raster_name = os.path.splitext(os.path.basename(raster_file_path))[0]
        
        # Define the output file name using the acronym
        output_filename = f"{acronym}_{raster_name}_processed.gpkg"
        output_filepath = os.path.join(PROCESSED_DATA_PATH, output_filename)

        if os.path.exists(output_filepath):
            print(f"Skipping {raster_name} for {municipality_name}. Processed file already exists.")
            continue

        print(f"  > Processing raster: {raster_name}")

        try:
            with rasterio.open(raster_file_path) as src:
                # Reproject the municipality boundary to match the raster's CRS
                municipio_gdf_raster_crs = municipio_gdf.to_crs(src.crs)
                
                # Clip the raster using the municipality's geometry
                print("    Clipping raster...")
                out_image, out_transform = rasterio.mask.mask(src, municipio_gdf_raster_crs.geometry, crop=True)
                
                # Use `rasterio.features.shapes` to convert raster to polygons
                print("    Converting raster to vector...")
                
                geometries = []
                values = []
                
                # The shapes generator yields (geometry, value) tuples
                shapes_gen = shapes(out_image[0], transform=out_transform)
                
                for geom, value in shapes_gen:
                    geometries.append(shape(geom))
                    values.append(value)
                
                # Create a temporary GeoDataFrame
                temp_gdf = gpd.GeoDataFrame({
                    'geometry': geometries,
                    'class': values
                }, crs=src.crs)
                
                # Filter out null or zero values
                temp_gdf = temp_gdf[temp_gdf['class'] != 0].copy()
                
                if temp_gdf.empty:
                    print("    No valid data found after filtering.")
                    continue
                
                # Reproject to the target CRS and dissolve
                print("    Reprojecting and dissolving by 'class'...")
                temp_gdf = temp_gdf.to_crs(TARGET_CRS)
                processed_gdf = temp_gdf.dissolve(by='class', as_index=False)
                
                # Calculate area in square kilometers
                print("    Calculating area in square kilometers...")
                processed_gdf['area_sqkm'] = processed_gdf.geometry.area / 10**6
                
                # Save the final GeoDataFrame
                print(f"    Saving processed data to {output_filepath}")
                processed_gdf.to_file(output_filepath, driver='GPKG')
                
                print(f"  > Processing complete for {raster_name} in {municipality_name}.")
                            
        except Exception as e:
            print(f"  > Error processing {raster_name}: {e}")

# Main execution loop
if __name__ == "__main__":
    # Create the processed data directory if it doesn't exist
    if not os.path.exists(PROCESSED_DATA_PATH):
        os.makedirs(PROCESSED_DATA_PATH)
        print(f"Created directory: {PROCESSED_DATA_PATH}")

    # Process data for each defined municipality
    for muni, acronym in MUNICIPALITIES.items():
        process_municipality_data(muni, acronym)

    print("\nAll processing complete.")

--- Processing data for Santa Cruz do Sul ---
Loading municipalities data...
  > Processing raster: mapbiomas_collection90_def_sec_veg_accumulated_v1-deforestation_accumulated_1986_2023
    Clipping raster...
    Converting raster to vector...
    Reprojecting and dissolving by 'class'...
    Calculating area in square kilometers...
    Saving processed data to ./data/processed/scs_mapbiomas_collection90_def_sec_veg_accumulated_v1-deforestation_accumulated_1986_2023_processed.gpkg
  > Processing complete for mapbiomas_collection90_def_sec_veg_accumulated_v1-deforestation_accumulated_1986_2023 in Santa Cruz do Sul.
  > Processing raster: brazil_coverage_2023
    Clipping raster...
    Converting raster to vector...
    Reprojecting and dissolving by 'class'...
    Calculating area in square kilometers...
    Saving processed data to ./data/processed/scs_brazil_coverage_2023_processed.gpkg
  > Processing complete for brazil_coverage_2023 in Santa Cruz do Sul.
  > Processing raster: brazi